In [1]:
# 라이브러리

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import xgboost as xgb

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from category_encoders import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 데이터 로드

orders = pd.read_csv('orders.csv' )
order_products_train = pd.read_csv('order_products_train.csv')
order_products_prior = pd.read_csv('order_products_prior.csv')
products = pd.read_csv('products.csv')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
train = pd.read_csv('train.csv')

In [3]:
# 데이터 재구성
# 카테고리로 (text 값 유한한 리스트를 표현할 때 사용)

aisles['aisle'] = aisles['aisle'].astype('category')

departments['department'] = departments['department'].astype('category')

orders['eval_set'] = orders['eval_set'].astype('category')

products['product_name'] = products['product_name'].astype('category')

In [4]:
# order + order_products_prior (이너조인)

op = orders.merge(order_products_prior, on='order_id', how='inner')
op.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [5]:
# 사용자 예측 변수 (고객 당 총 주문 수)
# user (user_id로 그룹 만들고 order_number 높은 순 / u_total_orders로 넣어)

user = op.groupby('user_id')['order_number'].max().to_frame('u_total_orders')
user.head()

,u_total_orders
user_id,
1,10
2,14
3,12
4,5
5,4


In [6]:
# user_id 인덱스 --> 열로 바꿈

user = user.reset_index()
user.head()

,user_id,u_total_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [7]:
# 고객이 제품을 재주문한 빈도
# u_reorder (user_id로 그룹 만들고 reordered 평균 / u_reordered_ratio로 넣어)

u_reorder = op.groupby('user_id')['reordered'].mean().to_frame('u_reordered_ratio')
u_reorder = u_reorder.reset_index()
u_reorder.head()

,user_id,u_reordered_ratio
0,1,0.694915
1,2,0.476923
2,3,0.625000
3,4,0.055556
4,5,0.378378


In [8]:
# user 랑 u_reorder랑 머지

user = user.merge(u_reorder, on='user_id', how='left')

#del u_reorder
#gc.collect()

user.head()

,user_id,u_total_orders,u_reordered_ratio
0,1,10,0.694915
1,2,14,0.476923
2,3,12,0.625000
3,4,5,0.055556
4,5,4,0.378378


In [9]:
# 각 제품에 대한 총 구매 수
# u_reorder (product_id로 그룹 만들고 order_id 카운트 / p_total_purchases로 넣어)
prd = op.groupby('product_id')['order_id'].count().to_frame('p_total_purchases')
prd.head()

,p_total_purchases
product_id,
1,1852
2,90
3,277
4,329
5,15


In [10]:
# product_id 인덱스 --> 열로 바꿈

prd = prd.reset_index()
prd.head()

,product_id,p_total_purchases
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [11]:
# .product_id로 그룹 만들고 .filter() 사용해서 40개 이상의 행이 있는 그룹만 유지  ( <-- 구매 횟수 40회 미만 제거)
# x on lambda 함수는 각 그룹을 나타내는 임시 변수
# DataFrame의 shape[0]은 행 수를 반환

p_reorder = op.groupby('product_id').filter(lambda x: x.shape[0] >40)
p_reorder.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [12]:
# 그룹 제품, 재주문 평균 계산
# p_reorder (product_id로 그룹 만들고 reordered 평균 / p_reorder_ratio로 넣어)

p_reorder = p_reorder.groupby('product_id')['reordered'].mean().to_frame('p_reorder_ratio')
p_reorder = p_reorder.reset_index()
p_reorder.head()

,product_id,p_reorder_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,8,0.503030


In [13]:
# prd + p_reorder 조인

prd = prd.merge(p_reorder, on='product_id', how='left')
prd.head()

,product_id,p_total_purchases,p_reorder_ratio
0,1,1852,0.613391
1,2,90,0.133333
2,3,277,0.732852
3,4,329,0.446809
4,5,15,NaN


In [14]:
# NaN값 0으로

prd['p_reorder_ratio'] = prd['p_reorder_ratio'].fillna(value=0)
prd.head()

,product_id,p_total_purchases,p_reorder_ratio
0,1,1852,0.613391
1,2,90,0.133333
2,3,277,0.732852
3,4,329,0.446809
4,5,15,0.000000


In [15]:
# 사용자가 제품을 구매한 횟수
# p_reorder ('user_id', 'product_id'로 그룹 만들고 'order_id' 카운트 / uxp_total_bought로 넣어)
 
uxp = op.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('uxp_total_bought')
uxp.head()

uxp_total_bought
user_id product_id                  
1       196                       10
        10258                      9
        10326                      1
        12427                     10
        13032                      3

In [16]:
# user_id & product_id 인덱스 --> 열로 바꿈

uxp = uxp.reset_index()
uxp.head()

,user_id,product_id,uxp_total_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [17]:
times = op.groupby(['user_id', 'product_id'])[['order_id']].count()
times.columns = ['Times_Bought_N']
times.head()

Times_Bought_N
user_id product_id                
1       196                     10
        10258                    9
        10326                    1
        12427                   10
        13032                    3

In [18]:
total_orders = op.groupby('user_id')['order_number'].max().to_frame('total_orders')
total_orders.head()

,total_orders
user_id,
1,10
2,14
3,12
4,5
5,4


In [19]:
first_order_no = op.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order_number')
first_order_no  = first_order_no.reset_index()  # 인덱스 --> 열로 바꿈
first_order_no.head()

,user_id,product_id,first_order_number
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,2


In [20]:
# span (total_orders + first_order_no 머지)
span = pd.merge(total_orders, first_order_no, on='user_id', how='right')
span.head()

,user_id,total_orders,product_id,first_order_number
0,1,10,196,1
1,1,10,10258,2
2,1,10,10326,5
3,1,10,12427,1
4,1,10,13032,2


In [21]:
span['Order_Range_D'] = span.total_orders - span.first_order_number + 1  # (+1은 제품 구매한 첫 번째 주문 차이에 포함)
span.head()

,user_id,total_orders,product_id,first_order_number,Order_Range_D
0,1,10,196,1,10
1,1,10,10258,2,9
2,1,10,10326,5,6
3,1,10,12427,1,10
4,1,10,13032,2,9


In [22]:
# times, span 머지

uxp_ratio = pd.merge(times, span, on=['user_id', 'product_id'], how='left')
uxp_ratio.head()

,user_id,product_id,Times_Bought_N,total_orders,first_order_number,Order_Range_D
0,1,196,10,10,1,10
1,1,10258,9,10,2,9
2,1,10326,1,10,5,6
3,1,12427,10,10,1,10
4,1,13032,3,10,2,9


In [23]:
# 비율 (사용자 및 제품별)

uxp_ratio['uxp_reorder_ratio'] = uxp_ratio.Times_Bought_N / uxp_ratio.Order_Range_D
uxp_ratio.head()

,user_id,product_id,Times_Bought_N,total_orders,first_order_number,Order_Range_D,uxp_reorder_ratio
0,1,196,10,10,1,10,1.000000
1,1,10258,9,10,2,9,1.000000
2,1,10326,1,10,5,6,0.166667
3,1,12427,10,10,1,10,1.000000
4,1,13032,3,10,2,9,0.333333


In [24]:
# user_id, product_id, uxp_reorder_ratio만 남겨

uxp_ratio = uxp_ratio.drop(['Times_Bought_N', 'total_orders', 'first_order_number', 'Order_Range_D'], axis=1)
uxp_ratio.head()

,user_id,product_id,uxp_reorder_ratio
0,1,196,1.000000
1,1,10258,1.000000
2,1,10326,0.166667
3,1,12427,1.000000
4,1,13032,0.333333


In [25]:
uxp = uxp.merge(uxp_ratio, on=['user_id', 'product_id'], how='left')

#del uxp_ratio
uxp.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio
0,1,196,10,1.000000
1,1,10258,9,1.000000
2,1,10326,1,0.166667
3,1,12427,10,1.000000
4,1,13032,3,0.333333


In [26]:
# 고객 1 주문

op[op.user_id==1].head(45)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0
5,2398795,1,prior,2,3,7,15.0,196,1,1
6,2398795,1,prior,2,3,7,15.0,10258,2,0
7,2398795,1,prior,2,3,7,15.0,12427,3,1
8,2398795,1,prior,2,3,7,15.0,13176,4,0
9,2398795,1,prior,2,3,7,15.0,26088,5,1


In [27]:
# 사용자가 제품을 구매한 횟수
# p_reorder ('user_id'로 그룹 만들고 order_number 의 가장 높은 수를 가져온 후 빼기(-) op.order_number를 사용하여 각 행의 order_number를 불러와

op['order_number_back'] = op.groupby('user_id')['order_number'].transform(max) - op.order_number +1 
op.head(15)  # 15개만 보자

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
0,2539329,1,prior,1,2,8,NaN,196,1,0,10
1,2539329,1,prior,1,2,8,NaN,14084,2,0,10
2,2539329,1,prior,1,2,8,NaN,12427,3,0,10
3,2539329,1,prior,1,2,8,NaN,26088,4,0,10
4,2539329,1,prior,1,2,8,NaN,26405,5,0,10
5,2398795,1,prior,2,3,7,15.0,196,1,1,9
6,2398795,1,prior,2,3,7,15.0,10258,2,0,9
7,2398795,1,prior,2,3,7,15.0,12427,3,1,9
8,2398795,1,prior,2,3,7,15.0,13176,4,0,9
9,2398795,1,prior,2,3,7,15.0,26088,5,1,9


In [28]:
# 각 고객의 마지막 5개 주문만 유지

op5 = op[op.order_number_back <= 5]
op5.head(15)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
29,3367565,1,prior,6,2,7,19.0,196,1,1,5
30,3367565,1,prior,6,2,7,19.0,12427,2,1,5
31,3367565,1,prior,6,2,7,19.0,10258,3,1,5
32,3367565,1,prior,6,2,7,19.0,25133,4,1,5
33,550135,1,prior,7,1,9,20.0,196,1,1,4
34,550135,1,prior,7,1,9,20.0,10258,2,1,4
35,550135,1,prior,7,1,9,20.0,12427,3,1,4
36,550135,1,prior,7,1,9,20.0,25133,4,1,4
37,550135,1,prior,7,1,9,20.0,13032,5,1,4
38,3108588,1,prior,8,1,14,14.0,12427,1,1,3


In [29]:
# 각 사용자에 대한 마지막 5개의 주문을 유지한 후 user_id, product_id로 묶어 
# .count( ) 사용해서 각 고객이 제품을 구매한 횟수

last_five = op5.groupby(['user_id','product_id'])[['order_id']].count()
last_five.columns = ['times_last5']
last_five.head(10)

times_last5
user_id product_id             
1       196                   5
        10258                 5
        12427                 5
        13032                 2
        25133                 5
        35951                 1
        38928                 1
        39657                 1
        46149                 3
        49235                 2

In [30]:
# uxp + last_five 머지

uxp = uxp.merge(last_five, on=['user_id', 'product_id'], how='left')
uxp.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5
0,1,196,10,1.000000,5.0
1,1,10258,9,1.000000,5.0
2,1,10326,1,0.166667,NaN
3,1,12427,10,1.000000,5.0
4,1,13032,3,0.333333,2.0


In [31]:
# NaN값 0으로

uxp = uxp.fillna(0)
uxp.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5
0,1,196,10,1.000000,5.0
1,1,10258,9,1.000000,5.0
2,1,10326,1,0.166667,0.0
3,1,12427,10,1.000000,5.0
4,1,13032,3,0.333333,2.0


In [32]:
# uxp + user 머지머지

data = uxp.merge(user, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio
0,1,196,10,1.000000,5.0,10,0.694915
1,1,10258,9,1.000000,5.0,10,0.694915
2,1,10326,1,0.166667,0.0,10,0.694915
3,1,12427,10,1.000000,5.0,10,0.694915
4,1,13032,3,0.333333,2.0,10,0.694915


In [33]:
# data + prd 머지

data = data.merge(prd, on='product_id', how='left')
data.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,p_total_purchases,p_reorder_ratio
0,1,196,10,1.000000,5.0,10,0.694915,35791,0.776480
1,1,10258,9,1.000000,5.0,10,0.694915,1946,0.713772
2,1,10326,1,0.166667,0.0,10,0.694915,5526,0.652009
3,1,12427,10,1.000000,5.0,10,0.694915,6476,0.740735
4,1,13032,3,0.333333,2.0,10,0.694915,3751,0.657158


In [34]:
# 두 단계에서 모든 고객의 향후 주문만 유지: 학습 및 테스트

orders_future = orders[((orders.eval_set=='train') | (orders.eval_set=='test'))]
orders_future = orders_future[ ['user_id', 'eval_set', 'order_id'] ]
orders_future.head(10)

,user_id,eval_set,order_id
10,1,train,1187899
25,2,train,1492625
38,3,test,2774568
44,4,test,329954
49,5,train,2196797
53,6,test,1528013
74,7,train,525192
78,8,train,880375
82,9,train,1094988
88,10,train,1822501


In [35]:
# data + orders_future 머지

data = data.merge(orders_future, on='user_id', how='left')
data.head(10)

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,p_total_purchases,p_reorder_ratio,eval_set,order_id
0,1,196,10,1.000000,5.0,10,0.694915,35791,0.776480,train,1187899
1,1,10258,9,1.000000,5.0,10,0.694915,1946,0.713772,train,1187899
2,1,10326,1,0.166667,0.0,10,0.694915,5526,0.652009,train,1187899
3,1,12427,10,1.000000,5.0,10,0.694915,6476,0.740735,train,1187899
4,1,13032,3,0.333333,2.0,10,0.694915,3751,0.657158,train,1187899
5,1,13176,2,0.222222,0.0,10,0.694915,379450,0.832555,train,1187899
6,1,14084,1,0.100000,0.0,10,0.694915,15935,0.810982,train,1187899
7,1,17122,1,0.166667,0.0,10,0.694915,13880,0.675576,train,1187899
8,1,25133,8,1.000000,5.0,10,0.694915,6196,0.740155,train,1187899
9,1,26088,2,0.200000,0.0,10,0.694915,2523,0.539041,train,1187899


In [36]:
dd = data.merge(order_products_train[['product_id','order_id', 'reordered']], on=['product_id','order_id'], how='left' )
dd.head(15)

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,p_total_purchases,p_reorder_ratio,eval_set,order_id,reordered
0,1,196,10,1.000000,5.0,10,0.694915,35791,0.776480,train,1187899,1.0
1,1,10258,9,1.000000,5.0,10,0.694915,1946,0.713772,train,1187899,1.0
2,1,10326,1,0.166667,0.0,10,0.694915,5526,0.652009,train,1187899,NaN
3,1,12427,10,1.000000,5.0,10,0.694915,6476,0.740735,train,1187899,NaN
4,1,13032,3,0.333333,2.0,10,0.694915,3751,0.657158,train,1187899,1.0
5,1,13176,2,0.222222,0.0,10,0.694915,379450,0.832555,train,1187899,NaN
6,1,14084,1,0.100000,0.0,10,0.694915,15935,0.810982,train,1187899,NaN
7,1,17122,1,0.166667,0.0,10,0.694915,13880,0.675576,train,1187899,NaN
8,1,25133,8,1.000000,5.0,10,0.694915,6196,0.740155,train,1187899,1.0
9,1,26088,2,0.200000,0.0,10,0.694915,2523,0.539041,train,1187899,1.0


In [37]:
# NaN값 0 처리

dd['reordered'] = dd['reordered'].fillna(0)
dd.head(15)

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,p_total_purchases,p_reorder_ratio,eval_set,order_id,reordered
0,1,196,10,1.000000,5.0,10,0.694915,35791,0.776480,train,1187899,1.0
1,1,10258,9,1.000000,5.0,10,0.694915,1946,0.713772,train,1187899,1.0
2,1,10326,1,0.166667,0.0,10,0.694915,5526,0.652009,train,1187899,0.0
3,1,12427,10,1.000000,5.0,10,0.694915,6476,0.740735,train,1187899,0.0
4,1,13032,3,0.333333,2.0,10,0.694915,3751,0.657158,train,1187899,1.0
5,1,13176,2,0.222222,0.0,10,0.694915,379450,0.832555,train,1187899,0.0
6,1,14084,1,0.100000,0.0,10,0.694915,15935,0.810982,train,1187899,0.0
7,1,17122,1,0.166667,0.0,10,0.694915,13880,0.675576,train,1187899,0.0
8,1,25133,8,1.000000,5.0,10,0.694915,6196,0.740155,train,1187899,1.0
9,1,26088,2,0.200000,0.0,10,0.694915,2523,0.539041,train,1187899,1.0


In [44]:
# 데이터 수가 너무 많아서 얘는 못함
# ddd = dd.pivot_table(index = 'user_id', columns = 'product_id')['p_reorder_ratio']

In [38]:
dd

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,p_total_purchases,p_reorder_ratio,eval_set,order_id,reordered
0,1,196,10,1.000000,5.0,10,0.694915,35791,0.776480,train,1187899,1.0
1,1,10258,9,1.000000,5.0,10,0.694915,1946,0.713772,train,1187899,1.0
2,1,10326,1,0.166667,0.0,10,0.694915,5526,0.652009,train,1187899,0.0
3,1,12427,10,1.000000,5.0,10,0.694915,6476,0.740735,train,1187899,0.0
4,1,13032,3,0.333333,2.0,10,0.694915,3751,0.657158,train,1187899,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13307948,206209,43961,3,0.300000,1.0,13,0.472868,55371,0.630583,train,272231,0.0
13307949,206209,44325,1,0.142857,0.0,13,0.472868,3485,0.401148,train,272231,0.0
13307950,206209,48370,1,0.333333,1.0,13,0.472868,3934,0.699288,train,272231,0.0
13307951,206209,48697,1,0.142857,0.0,13,0.472868,9783,0.357661,train,272231,0.0


In [39]:
dd2 = dd.head(50000)  # 5만개로 줄여서 다시
ddd = dd2.pivot_table(index = 'user_id', columns = 'product_id')['p_reorder_ratio']

C:\Users\dawn\AppData\Local\Temp\ipykernel_9592\4266198628.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ddd = dd2.pivot_table(index = 'user_id', columns = 'product_id')['p_reorder_ratio']


In [40]:
ddd

product_id,1,9,10,13,23,25,28,29,30,32,...,49667,49668,49670,49674,49677,49678,49679,49680,49682,49683
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,0.448502,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.691702
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.691702
810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
ddd.shape

(812, 13726)

In [59]:
# products.rename(columns = {'id': 'product_id'}, inplace = True)

In [66]:
# product_id로 머지한 ratio 만듦
# ratio = pd.merge(dd, products, on = 'product_id')  

In [45]:
# 제품 id 말고 제품 이름을 열로 넣으려고 했는데 메모리 너무 잡아먹어서 포기

# ratio_2 = ratio.head(5000)
# ratio_3 = ratio_2.pivot_table('p_reorder_ratio', index = 'user_id', columns = 'product_name').fillna(0)

In [42]:
# item이 row가 되어야 함 --> cosine similarity를 구할 때 row 기반으로 유사도를 측정하기 때문

ddd = ddd.transpose()
ddd

user_id,1,2,3,4,5,6,7,8,9,10,...,803,804,805,806,807,808,809,810,811,812
product_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,0.448502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# NaN값 없애

ddd = ddd.fillna(0)
ddd

user_id,1,2,3,4,5,6,7,8,9,10,...,803,804,805,806,807,808,809,810,811,812
product_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
9,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
10,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
13,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
23,0.0,0.448502,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49678,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
49679,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
49680,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [44]:
'''
" 코사인 유사도 !!! "

현재, row가 product_id, col이 user_id로 되어있음

아이템 기반 협업 필터링은 ~ 상품을 구매한 고객들은 다음 상품도 구매했다. 라는 뜻
즉, ~ 제품을 산 고객들은 이런 제품도 샀다. 라고 해석

그리고 그 기반은 평점이 비슷한 것을 기반으로 함
이제 그 평점이 비슷하다는 것을 코사인 유사도로 측정해서 확인
'''

item_based_collabor = cosine_similarity(ddd)
item_based_collabor

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.03910309],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.09578263],
       [0.        , 0.        , 0.03910309, ..., 0.        , 0.09578263,
        1.        ]])

In [45]:
print(ddd.shape)
print(item_based_collabor.shape)

(13726, 812)
(13726, 13726)


In [46]:
item_based_collabor = pd.DataFrame(data = item_based_collabor, index = ddd.index, columns = ddd.index)

In [47]:
item_based_collabor.head()

product_id,1,9,10,13,23,25,28,29,30,32,...,49667,49668,49670,49674,49677,49678,49679,49680,49682,49683
product_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039103
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
23,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [48]:
# 특정 item에 있어 비슷한 item을 추천해주는 기능을 넣어보자
# 제품 id가 아니라 제품명이 떴으면 좋겠는데 컴퓨터 메모리가 못버틴다..ㅠ

def get_item_based_collabor(product_id):
    return item_based_collabor[product_id].sort_values(ascending=False)[:6]  # 6개까지

In [69]:
get_item_based_collabor(1) # 스낵

C:\Users\dawn\AppData\Local\Temp\ipykernel_24108\516832777.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return item_based_collabor[product_id].sort_values(ascending=False)[:6]


product_id
1        1.000000
22340    0.707107
48667    0.577350
30553    0.500000
14177    0.500000
1519     0.500000
Name: 1, dtype: float64

In [72]:
get_item_based_collabor(9)  # 요거트

C:\Users\dawn\AppData\Local\Temp\ipykernel_24108\516832777.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return item_based_collabor[product_id].sort_values(ascending=False)[:6]


product_id
47242    1.0
19189    1.0
7069     1.0
26727    1.0
11636    1.0
9        1.0
Name: 9, dtype: float64

In [56]:
get_item_based_collabor(23) # 버거(먹을 거)

C:\Users\dawn\AppData\Local\Temp\ipykernel_9592\1645639962.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return item_based_collabor[product_id].sort_values(ascending=False)[:6]  # 6개까지


product_id
23       1.00000
28874    0.57735
15841    0.50000
14306    0.50000
48964    0.50000
31833    0.50000
Name: 23, dtype: float64

In [57]:
get_item_based_collabor(25) # 프로틴바

C:\Users\dawn\AppData\Local\Temp\ipykernel_9592\1645639962.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return item_based_collabor[product_id].sort_values(ascending=False)[:6]  # 6개까지


product_id
25       1.000000
6902     0.632456
21252    0.632456
43011    0.447214
305      0.447214
7882     0.447214
Name: 25, dtype: float64